# Scraping Premier League Table (Training Data)

## Overview

This notebook aims to scrape data from the Premier League table from sofascore to gather training data for our betting prediction model. The extracted data will be utilized to train our model for accurate predictions. The primary objectives of this notebook include:

1. **Data Extraction**: Retrieve relevant information from the Premier League table, including team standings, recent match results, and performance statistics.
2. **Data Processing**: Cleanse and format the scraped data to ensure consistency and accuracy for model training.
3. **Database Integration**: Store the processed data in the previous_league_standings table which we would later merge with completed_matches table and append to our `training_data` table within our database for future model training sessions.

## Requirements

- Python 3.x
- Libraries: `selenium web driver`, `pandas`, `sqlalchemy`
- Access to the database containing the `training_data` table

## Notes

- Prior to execution, ensure that database credentials are correctly configured.
- This notebook should be executed periodically to keep the completed_matches up-to-date with the previous Premier League data. to ensure this, run this notebook before you run `Forebet Completed Matches`

## Updates July 2024
### Project Migration and Update Process
- Migration: Migrated project from Jupyter Notebook 6 to Notebook 7.
- Python Version: Updated from Python 3.9 to Python 3.12.
- Selenium Version and Usage: Noted that with Selenium 4.2, it was possible to use webdriver without specifying service and options AND PATH, eliminating unnecessary code.

### Web Updates November 2024
- Sofascore updated the information on their websites.
- They added a DIFF column which i had to consider during scraping. However since this project started with without this column in mind, i drop that column before passing the data into the db. 

In [ ]:
# let's install the necessary libraries
!pip install pandas sqlalchemy

In [ ]:
# Make sure youre using latest version of selenium to prevent running into errors. 
# Make sure your chrome driver version is compatible with your Chrome versions. 
# Use this link to update to latest Chrome and chromerdriver https://googlechromelabs.github.io/chrome-for-testing/ if necessary.
!pip install --upgrade selenium

In [1]:
from selenium import webdriver
import pandas as pd
from sqlalchemy import create_engine
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Data Scraping

In [24]:

team_labels = {
        'Arsenal': 1,
        'Aston Villa': 2,
        'Bournemouth': 3,
        'Brentford': 4,
        'Brighton': 5,
        'Burnley': 6,
        'Chelsea': 7,
        'Crystal Palace': 8,
        'Everton': 9,
        'Fulham': 10,
        'Ipswich': 11,
        'Leeds Utd': 12,
        'Leicester': 13,
        'Liverpool': 14,
        'Man City': 15,
        'Man Utd': 16,
        'Newcastle': 17,
        'Norwich': 18,
        'Sheffield': 19,
        'Southampton': 20,
        'Tottenham': 21,
        'West Ham': 22,
        'Luton': 23,
        'Wolves': 24,
        'Sheffield Utd': 25,
        'Forest': 26
    }


In [8]:
# Initialize WebDriver 
# Set up the Chrome driver
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Opens browser in full-screen
driver = webdriver.Chrome(options=chrome_options)

# Navigate to the specified URL
driver.get('https://www.sofascore.com/tournament/football/england/premier-league/17#id:61627')

# Add a wait to ensure the element is present
try:
    # Wait for the consent pop-up and click the consent button
    consent_button = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable((By.XPATH, "//p[@class='fc-button-label' and text()='Consent']"))
    )
    consent_button.click()
    print("Clicked the consent button!")
except Exception as e:
    print(f"Error: {e}")

# Now find the fixture containers within the body element
league_table_results = driver.find_elements("class name", "eHXJll")
print(league_table_results)

Clicked the consent button!
[<selenium.webdriver.remote.webelement.WebElement (session="c600f3873b21bebd594f0e61df907550", element="f.E8EA97A07C051922E5D1F11BA5DCD148.d.74E4E6D781E19AF39F60CF9FA391DAE7.e.419")>]


In [9]:
league_table_results_container = []

for results in league_table_results:
    # Extract the text from the fixture container
    results_text = results.text
    league_table_results_container.append(results_text)
    print(results_text)

ALLHOMEAWAY
#
Team
P
W
D
L
DIFF
Goals
Last 5
PTS
1
Liverpool
11
9
1
1
+15
21:6
W
W
D
W
W
28
2
Man City
11
7
2
2
+9
22:13
W
W
W
L
L
23
3
Chelsea
11
5
4
2
+8
21:13
D
L
W
D
D
19
4
Arsenal
11
5
4
2
+6
18:12
W
L
D
L
D
19
5
Forest
11
5
4
2
+5
15:10
D
W
W
W
L
19
6
Brighton
11
5
4
2
+4
19:15
W
W
D
L
W
19
7
Fulham
11
5
3
3
+3
16:13
L
L
D
W
W
18
8
Newcastle
11
5
3
3
+2
13:11
D
L
L
W
W
18
9
Aston Villa
11
5
3
3
0
17:17
D
W
D
L
L
18
10
Tottenham
11
5
1
5
+10
23:13
L
W
L
W
L
16
11
Brentford
11
5
1
5
0
22:22
W
L
W
L
W
16
12
Bournemouth
11
4
3
4
0
15:15
L
W
D
W
L
15
13
Man Utd
11
4
3
4
0
12:12
D
W
L
D
W
15
14
West Ham
11
3
3
5
-6
13:19
W
L
W
L
D
12
15
Leicester
11
2
4
5
-7
14:21
W
W
L
D
L
10
16
Everton
11
2
4
5
-7
10:17
D
W
D
L
D
10
17
Ipswich
11
1
5
5
-10
12:22
L
L
L
D
W
8
18
Crystal Palace
11
1
4
6
-7
8:15
L
L
W
D
L
7
19
Wolves
11
1
3
7
-11
16:27
L
L
D
D
W
6
20
Southampton
11
1
1
9
-14
7:21
L
L
L
W
L
4


#### Load the Scraped Data and inspect before quitting.

In [10]:
league_table_results_container

['ALLHOMEAWAY\n#\nTeam\nP\nW\nD\nL\nDIFF\nGoals\nLast 5\nPTS\n1\nLiverpool\n11\n9\n1\n1\n+15\n21:6\nW\nW\nD\nW\nW\n28\n2\nMan City\n11\n7\n2\n2\n+9\n22:13\nW\nW\nW\nL\nL\n23\n3\nChelsea\n11\n5\n4\n2\n+8\n21:13\nD\nL\nW\nD\nD\n19\n4\nArsenal\n11\n5\n4\n2\n+6\n18:12\nW\nL\nD\nL\nD\n19\n5\nForest\n11\n5\n4\n2\n+5\n15:10\nD\nW\nW\nW\nL\n19\n6\nBrighton\n11\n5\n4\n2\n+4\n19:15\nW\nW\nD\nL\nW\n19\n7\nFulham\n11\n5\n3\n3\n+3\n16:13\nL\nL\nD\nW\nW\n18\n8\nNewcastle\n11\n5\n3\n3\n+2\n13:11\nD\nL\nL\nW\nW\n18\n9\nAston Villa\n11\n5\n3\n3\n0\n17:17\nD\nW\nD\nL\nL\n18\n10\nTottenham\n11\n5\n1\n5\n+10\n23:13\nL\nW\nL\nW\nL\n16\n11\nBrentford\n11\n5\n1\n5\n0\n22:22\nW\nL\nW\nL\nW\n16\n12\nBournemouth\n11\n4\n3\n4\n0\n15:15\nL\nW\nD\nW\nL\n15\n13\nMan Utd\n11\n4\n3\n4\n0\n12:12\nD\nW\nL\nD\nW\n15\n14\nWest Ham\n11\n3\n3\n5\n-6\n13:19\nW\nL\nW\nL\nD\n12\n15\nLeicester\n11\n2\n4\n5\n-7\n14:21\nW\nW\nL\nD\nL\n10\n16\nEverton\n11\n2\n4\n5\n-7\n10:17\nD\nW\nD\nL\nD\n10\n17\nIpswich\n11\n1\n5\n5\n-10\n12:2

In [11]:
driver.quit()

# Data Cleaning 

In [12]:
# Extract the relevant part of the string
data_string = league_table_results_container.copy()[0]

# Find the index where the data rows start (`\n1`)
start_data_index = data_string.find("\n1")

# Split into columns and data using the index
df_columns = data_string[:start_data_index].split("\n")  # Everything before `\n1`
data = data_string[start_data_index+1:].split("\n")      # Everything from `\n1` onwards

# Clean up any empty entries caused by splitting
df_columns = [col for col in df_columns if col]
data = [item for item in data if item]

# Print results
print("Columns:", df_columns)
print("\nData:", data)

Columns: ['ALLHOMEAWAY', '#', 'Team', 'P', 'W', 'D', 'L', 'DIFF', 'Goals', 'Last 5', 'PTS']

Data: ['1', 'Liverpool', '11', '9', '1', '1', '+15', '21:6', 'W', 'W', 'D', 'W', 'W', '28', '2', 'Man City', '11', '7', '2', '2', '+9', '22:13', 'W', 'W', 'W', 'L', 'L', '23', '3', 'Chelsea', '11', '5', '4', '2', '+8', '21:13', 'D', 'L', 'W', 'D', 'D', '19', '4', 'Arsenal', '11', '5', '4', '2', '+6', '18:12', 'W', 'L', 'D', 'L', 'D', '19', '5', 'Forest', '11', '5', '4', '2', '+5', '15:10', 'D', 'W', 'W', 'W', 'L', '19', '6', 'Brighton', '11', '5', '4', '2', '+4', '19:15', 'W', 'W', 'D', 'L', 'W', '19', '7', 'Fulham', '11', '5', '3', '3', '+3', '16:13', 'L', 'L', 'D', 'W', 'W', '18', '8', 'Newcastle', '11', '5', '3', '3', '+2', '13:11', 'D', 'L', 'L', 'W', 'W', '18', '9', 'Aston Villa', '11', '5', '3', '3', '0', '17:17', 'D', 'W', 'D', 'L', 'L', '18', '10', 'Tottenham', '11', '5', '1', '5', '+10', '23:13', 'L', 'W', 'L', 'W', 'L', '16', '11', 'Brentford', '11', '5', '1', '5', '0', '22:22', 'W', 

### Handling Goal Difference. 
After an update on the website. I added some methods for including goal difference 
- The prompt for weekly round, is to help handle weekly rounds below 5. If its not the beginning of the season any round above 5 will work the same.

In [13]:
# Dynamic weekly round: specify the current match week
weekly_round = int(input("Enter the current weekly round (matches played so far): "))

# Constants based on data structure
BASE_TEAM_STATS = 8  # Base stats for each team: rank, team name, P, W, D, L, DIFF
MAX_LAST_MATCHES = 5  # Maximum match results to include in 'Last 5'
POINTS_FIELD = 1  # Points field length

# Calculate slice length: base stats + current match results (up to MAX_LAST_MATCHES) + points field
slice_length = BASE_TEAM_STATS + min(weekly_round, MAX_LAST_MATCHES) + POINTS_FIELD

# Split the data into slices for each team
data_array = [data[i:i+slice_length] for i in range(0, len(data), slice_length)]

# Print each team's data for verification
for team_data in data_array:
    
    print(team_data)


Enter the current weekly round (matches played so far):  11


['1', 'Liverpool', '11', '9', '1', '1', '+15', '21:6', 'W', 'W', 'D', 'W', 'W', '28']
['2', 'Man City', '11', '7', '2', '2', '+9', '22:13', 'W', 'W', 'W', 'L', 'L', '23']
['3', 'Chelsea', '11', '5', '4', '2', '+8', '21:13', 'D', 'L', 'W', 'D', 'D', '19']
['4', 'Arsenal', '11', '5', '4', '2', '+6', '18:12', 'W', 'L', 'D', 'L', 'D', '19']
['5', 'Forest', '11', '5', '4', '2', '+5', '15:10', 'D', 'W', 'W', 'W', 'L', '19']
['6', 'Brighton', '11', '5', '4', '2', '+4', '19:15', 'W', 'W', 'D', 'L', 'W', '19']
['7', 'Fulham', '11', '5', '3', '3', '+3', '16:13', 'L', 'L', 'D', 'W', 'W', '18']
['8', 'Newcastle', '11', '5', '3', '3', '+2', '13:11', 'D', 'L', 'L', 'W', 'W', '18']
['9', 'Aston Villa', '11', '5', '3', '3', '0', '17:17', 'D', 'W', 'D', 'L', 'L', '18']
['10', 'Tottenham', '11', '5', '1', '5', '+10', '23:13', 'L', 'W', 'L', 'W', 'L', '16']
['11', 'Brentford', '11', '5', '1', '5', '0', '22:22', 'W', 'L', 'W', 'L', 'W', '16']
['12', 'Bournemouth', '11', '4', '3', '4', '0', '15:15', 'L', '

### Detecting ':' in the array and then split it
I updated the process here. I like to keep Old code, to show how goofy i was.

In [14]:
for data in data_array:
    for d in data:
         if d.__contains__(':'):
             data[data.index(d):data.index(d)+1] = d.split(':')

print(data_array)

[['1', 'Liverpool', '11', '9', '1', '1', '+15', '21', '6', 'W', 'W', 'D', 'W', 'W', '28'], ['2', 'Man City', '11', '7', '2', '2', '+9', '22', '13', 'W', 'W', 'W', 'L', 'L', '23'], ['3', 'Chelsea', '11', '5', '4', '2', '+8', '21', '13', 'D', 'L', 'W', 'D', 'D', '19'], ['4', 'Arsenal', '11', '5', '4', '2', '+6', '18', '12', 'W', 'L', 'D', 'L', 'D', '19'], ['5', 'Forest', '11', '5', '4', '2', '+5', '15', '10', 'D', 'W', 'W', 'W', 'L', '19'], ['6', 'Brighton', '11', '5', '4', '2', '+4', '19', '15', 'W', 'W', 'D', 'L', 'W', '19'], ['7', 'Fulham', '11', '5', '3', '3', '+3', '16', '13', 'L', 'L', 'D', 'W', 'W', '18'], ['8', 'Newcastle', '11', '5', '3', '3', '+2', '13', '11', 'D', 'L', 'L', 'W', 'W', '18'], ['9', 'Aston Villa', '11', '5', '3', '3', '0', '17', '17', 'D', 'W', 'D', 'L', 'L', '18'], ['10', 'Tottenham', '11', '5', '1', '5', '+10', '23', '13', 'L', 'W', 'L', 'W', 'L', '16'], ['11', 'Brentford', '11', '5', '1', '5', '0', '22', '22', 'W', 'L', 'W', 'L', 'W', '16'], ['12', 'Bournemout

In [15]:
### Very neccessary to double check the Last 5 games starting and ending indeces. Update if Website changes
data_array[0][9:slice_length]

['W', 'W', 'D', 'W', 'W']

In [16]:
def calculate_ppg_and_create_last_5_matches(two_d_array):
    """
    Calculate the points per game (PPG) and create a string representing the last 5 matches for each team.

    Args:
        two_d_array (list of lists): Two-dimensional array containing match data for each team.

    Returns:
        list of lists: Updated two-dimensional array with additional columns for last 5 matches and PPG.
    """
    for row in two_d_array:
        ppg = 0
        for result in row[9:slice_length-1]:
            if result == 'W':
                ppg += 3
            elif result == 'D':
                ppg += 1
            elif result == 'L':
                ppg += 0
        last_5_matches = ''.join(row[9:slice_length])  # Concatenate match results from index 7 to 11
        row[9:slice_length] = [last_5_matches, ppg / 5]  # Replace old match results with last 5 matches string and PPG
    
    return two_d_array

In [17]:
premier_league_columns  = ['pos', 'team', 'pld', 'wins', 'draws', 'losses', 'diff' , 'gf', 'ga', 'last_5_matches', 'ppg_last_5_matches', 'points']


In [18]:
split_data = data_array.copy()

In [19]:
df_data = calculate_ppg_and_create_last_5_matches(split_data)
df_data

[['1', 'Liverpool', '11', '9', '1', '1', '+15', '21', '6', 'WWDWW', 2.0, '28'],
 ['2', 'Man City', '11', '7', '2', '2', '+9', '22', '13', 'WWWLL', 1.8, '23'],
 ['3', 'Chelsea', '11', '5', '4', '2', '+8', '21', '13', 'DLWDD', 1.0, '19'],
 ['4', 'Arsenal', '11', '5', '4', '2', '+6', '18', '12', 'WLDLD', 0.8, '19'],
 ['5', 'Forest', '11', '5', '4', '2', '+5', '15', '10', 'DWWWL', 2.0, '19'],
 ['6', 'Brighton', '11', '5', '4', '2', '+4', '19', '15', 'WWDLW', 1.4, '19'],
 ['7', 'Fulham', '11', '5', '3', '3', '+3', '16', '13', 'LLDWW', 0.8, '18'],
 ['8', 'Newcastle', '11', '5', '3', '3', '+2', '13', '11', 'DLLWW', 0.8, '18'],
 ['9',
  'Aston Villa',
  '11',
  '5',
  '3',
  '3',
  '0',
  '17',
  '17',
  'DWDLL',
  1.0,
  '18'],
 ['10',
  'Tottenham',
  '11',
  '5',
  '1',
  '5',
  '+10',
  '23',
  '13',
  'LWLWL',
  1.2,
  '16'],
 ['11', 'Brentford', '11', '5', '1', '5', '0', '22', '22', 'WLWLW', 1.2, '16'],
 ['12',
  'Bournemouth',
  '11',
  '4',
  '3',
  '4',
  '0',
  '15',
  '15',
  'LWDWL

In [20]:
# Create DataFrame
premier_league_table = pd.DataFrame(df_data, columns=premier_league_columns)
print(premier_league_table)


   pos            team pld wins draws losses diff  gf  ga last_5_matches  \
0    1       Liverpool  11    9     1      1  +15  21   6          WWDWW   
1    2        Man City  11    7     2      2   +9  22  13          WWWLL   
2    3         Chelsea  11    5     4      2   +8  21  13          DLWDD   
3    4         Arsenal  11    5     4      2   +6  18  12          WLDLD   
4    5          Forest  11    5     4      2   +5  15  10          DWWWL   
5    6        Brighton  11    5     4      2   +4  19  15          WWDLW   
6    7          Fulham  11    5     3      3   +3  16  13          LLDWW   
7    8       Newcastle  11    5     3      3   +2  13  11          DLLWW   
8    9     Aston Villa  11    5     3      3    0  17  17          DWDLL   
9   10       Tottenham  11    5     1      5  +10  23  13          LWLWL   
10  11       Brentford  11    5     1      5    0  22  22          WLWLW   
11  12     Bournemouth  11    4     3      4    0  15  15          LWDWL   
12  13      

In [21]:
premier_league_table.drop(columns=['diff'], inplace=True)

In [22]:
def team_to_label(team_name):
    return team_labels.get(team_name)

In [25]:
premier_league_table['team'] = premier_league_table['team'].map(team_to_label)

In [26]:
premier_league_table

,pos,team,pld,wins,draws,losses,gf,ga,last_5_matches,ppg_last_5_matches,points
0,1,14,11,9,1,1,21,6,WWDWW,2.0,28
1,2,15,11,7,2,2,22,13,WWWLL,1.8,23
2,3,7,11,5,4,2,21,13,DLWDD,1.0,19
3,4,1,11,5,4,2,18,12,WLDLD,0.8,19
4,5,26,11,5,4,2,15,10,DWWWL,2.0,19
5,6,5,11,5,4,2,19,15,WWDLW,1.4,19
6,7,10,11,5,3,3,16,13,LLDWW,0.8,18
7,8,17,11,5,3,3,13,11,DLLWW,0.8,18
8,9,2,11,5,3,3,17,17,DWDLL,1.0,18
9,10,21,11,5,1,5,23,13,LWLWL,1.2,16


#### Confirm its the right Weekly Round in order to join with the right dataset. 
Note this data will eb considered as 'Previous League standings'(The data we are scrapping will be vital before the weekly fixtures | completed_matches in our case. Because we need the stats before the matches are played

In [27]:
Round  = 'Round ' + ' ' +  premier_league_table['pld'].mode()
Round

0    Round  11
Name: pld, dtype: object

Establish your DB 
Make sure you have already created a database ready for it. In my next update i will have automations for creating the databse for you

In [28]:
# Database connection
user = 'test_user'
password = 'password'
host = 'localhost'
port = 3306
database = 'bet_tips_validation_web_app'

In [29]:
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{database}')
premier_league_table.to_sql('previous_week_league_standings', con=engine, if_exists='replace', index=False)

20